In [8]:
import sys
sys.path.append('..')
from tool import *

'''
Set the distance weights to [1, 1, 1, 1], 
search for the 20 nearest neighboring grids to the missing value, 
and return the albedo and distance vectors of the neighboring grids.
'''
def fill_albedo(m,type,landcover_type):
    max_distance=3
    distence_w_dim=[1,1,1,1]
    bands=['albedo_BSA_vis', 
        'albedo_WSA_vis', 
        'albedo_BSA_nir', 
        'albedo_WSA_nir', 
        'albedo_BSA_shortwave', 
        'albedo_WSA_shortwave']
    m_index=[(m-2)%12+1,(m-1)%12+1,(m)%12+1]
    offset=[1 if m_index[j]-m_index[0]>=0 else -1 for j in range(len(m_index))]
    if np.array(offset).sum()==len(offset):
        offset=[0]*len(m_index)
    vs=[]
    ms=[]
    land_proportions=[]
    for i in range(len(offset)):    
        NC=nc.Dataset(f'/data2/hzy/albedo2/albedo_information_nc/nan_to_2000/month_{m_index[i]}_{landcover_type}.nc','r')
        for band in bands:
            vs.append(NC.variables[f'{band}-{type}'][:])
        if type!='ocean':
            ms.append(NC.variables[f'area-{type}'][:])
        # qa=NC.variables['qa'][:]
        area_grid=NC.variables['area-grid'][:]
        area_land=NC.variables['area-land'][:]
        land_proportions.append(area_land/area_grid)
        NC.close()
    offset_bands=[]
    for i in offset:
        offset_bands+=[i]*len(bands)
    if offset[0]!=0:
        nan_array=np.zeros((1,180,360))*np.nan
        vs=[np.concatenate((nan_array,vs[i]),axis=0) if offset_bands[i]==1 else np.concatenate((vs[i],nan_array),axis=0) for i in range(len(offset_bands))]
        land_proportions=[np.concatenate((nan_array,land_proportions[i]),axis=0) if offset[i]==1 else np.concatenate((land_proportions[i],nan_array),axis=0) for i in range(len(offset))]
        if type!='ocean':
            ms=[np.concatenate((nan_array,ms[i]),axis=0) if offset[i]==1 else np.concatenate((ms[i],nan_array),axis=0) for i in range(len(offset))]
        else:
            ms=land_proportions  
    else:
        if type=='ocean':
            ms=land_proportions  
    M,H,W=vs[0].shape
    v=np.stack(vs).reshape((len(offset),len(bands),M,H,W)).transpose((1,0,2,3,4))#(6, 3, 21, 180, 360)
    m=np.stack(ms)

    all_indices = np.indices(v[0].shape).reshape(4, -1).T
    if type!='ocean':
        interp_indices = all_indices[((v[0]==2000) & (m>0)).reshape(-1)]
    else:
        interp_indices = all_indices[((v[0]==2000) & (m<1) & (m>0)).reshape(-1)]
    nonan_indices=all_indices[((~np.isnan(v[0])).reshape(-1)) & ((v[0]!=2000).reshape(-1))]
    # print(len(nonan_indices))
    interp_indices=[j for j in interp_indices if j[0]==int(len(offset)/2)]   

    kk=[search_nonan(v,interp_indice,max_distance=max_distance,distence_w_dim=distence_w_dim) for interp_indice in interp_indices]
    interp_values=np.array([a[0] for a in kk])
    dws=np.array([a[1] for a in kk])
    print('fill value '+str((1-np.isnan(interp_values).sum()/interp_values.shape[0])*100)+'%')
    dws_v=np.zeros(v.shape[1:])
    for z in range(len(interp_indices)):
        v[:,interp_indices[z][0],interp_indices[z][1],interp_indices[z][2],interp_indices[z][3]]=np.array(interp_values[z])
        dws_v[interp_indices[z][0],interp_indices[z][1],interp_indices[z][2],interp_indices[z][3]]=dws[z]
    y_pd=[]
    m_pd=[]
    type_pd=[]
    lat_pd=[]
    lon_pd=[]
    d1=[]
    d2=[]
    d3=[]
    d4=[]
    sub_band1=[]
    sub_band2=[]
    sub_band3=[]
    sub_band4=[]
    sub_band5=[]
    sub_band6=[]
    for z in range(len(interp_indices)):
        a=kk[z]
        for j in range(len(a[2])):
            if offset[int(len(offset)/2)]>0:
                y_pd.append(interp_indices[z][1]+2000)
                if interp_indices[z][1]==0:
                    print(interp_indices[z])
                    print(v.shape)
                    raise('error')
            else:
                y_pd.append(interp_indices[z][1]+2001)
            m_pd.append(m)
            type_pd.append(type)
            lat_pd.append(90-interp_indices[z][2])
            lon_pd.append(interp_indices[z][3]-180)
            d1.append(a[2][j][0])
            d2.append(a[2][j][1])
            d3.append(a[2][j][2])
            d4.append(a[2][j][3])
            sub_band1.append(a[3][0][j])
            sub_band2.append(a[3][1][j])
            sub_band3.append(a[3][2][j])
            sub_band4.append(a[3][3][j])
            sub_band5.append(a[3][4][j])
            sub_band6.append(a[3][5][j])
    pd_dict={'y':y_pd,
            'm':m_pd,
            'type':type_pd,
            'lat':lat_pd,
            'lon':lon_pd,
            'd1':d1,
            'd2':d2,
            'd3':d3,
            'd4':d4,
            'sub_band1':sub_band1,
            'sub_band2':sub_band2,
            'sub_band3':sub_band3,
            'sub_band4':sub_band4,
            'sub_band5':sub_band5,
            'sub_band6':sub_band6}
    pd_all=pd.DataFrame.from_dict(pd_dict)
    if offset[int(len(offset)/2)]>=0:
        return v[:,int(len(offset)/2),offset[int(len(offset)/2)]:],dws_v[int(len(offset)/2),offset[int(len(offset)/2)]:],pd_all
    else:
        return v[:,int(len(offset)/2),:-1],dws_v[int(len(offset)/2),:-1],pd_all
    
def TS_IDW_4d(v,interp_indice,max_distance,distence_w_dim):
    
        value_bands=v[:,zero(interp_indice[0]-int(max_distance/distence_w_dim[0])):interp_indice[0]+int(max_distance/distence_w_dim[0])+1,
                zero(interp_indice[1]-int(max_distance/distence_w_dim[1])):interp_indice[1]+int(max_distance/distence_w_dim[1])+1,
                zero(interp_indice[2]-int(max_distance/distence_w_dim[2])):interp_indice[2]+int(max_distance/distence_w_dim[2])+1,
                zero(interp_indice[3]-int(max_distance/distence_w_dim[3])):interp_indice[3]+int(max_distance/distence_w_dim[3])+1]
        value_bands[value_bands==2000]=np.nan
        value=value_bands[0]
        non_nan_indices = np.where(~np.isnan(value))

        non_nan_points = np.array(list(zip(non_nan_indices[0], non_nan_indices[1], non_nan_indices[2],non_nan_indices[3])))
         
        if len(non_nan_points)==0:
                interpolated_values=[np.nan]
                non_nan_values_bands=[]
                non_nan_points2=[]
                dws=np.nan
        else:
                k=np.array([zero(-interp_indice[0]+int(max_distance/distence_w_dim[0]))-int(max_distance/distence_w_dim[0]),
                        zero(-interp_indice[1]+int(max_distance/distence_w_dim[1]))-int(max_distance/distence_w_dim[1]),
                        zero(-interp_indice[2]+int(max_distance/distence_w_dim[2]))-int(max_distance/distence_w_dim[2]),
                        zero(-interp_indice[3]+int(max_distance/distence_w_dim[3]))-int(max_distance/distence_w_dim[3])])
                # print(k)
                # print(non_nan_points)
                non_nan_points+=k
                distance_s = np.array([np.linalg.norm(np.array(non_nan_points[i])*np.array(distence_w_dim)) for i in range(len(non_nan_points))])
                if len(distance_s)<20:
                        interpolated_values=[np.nan]
                        non_nan_values_bands=[]
                        non_nan_points2=[]
                        dws=np.nan
                else:

                        
                        kth = np.partition(distance_s, 19)[19]
                        distance1=distance_s.copy()
                        distance1=distance1[distance1<=kth]
                        non_nan_points2=[non_nan_points[j] for j in range(len(distance_s)) if distance_s[j]<=kth]
                        weights = 1.0 / distance1**2
                        dws=np.sum(weights)
                        weights /= dws
                        interpolated_values=[]
                        non_nan_values_bands=[]
                        for i in range(value_bands.shape[0]):
                                distance=distance_s.copy()
                                value=value_bands[i]
                                non_nan_values = value[non_nan_indices]
                                non_nan_values2=np.array([non_nan_values[i] for i in range(len(distance)) if distance[i]<=kth])
                                non_nan_values_bands.append(non_nan_values2)
                                interpolated_value = np.sum(weights * non_nan_values2)
                                interpolated_values.append(interpolated_value)

        return interpolated_values,dws,non_nan_points2,non_nan_values_bands
    
def search_nonan(v,interp_indice,max_distance,distence_w_dim):
    search_n=1
    k,dws,non_nan_points2,non_nan_values_bands=TS_IDW_4d(v,interp_indice,max_distance=max_distance*search_n,distence_w_dim=distence_w_dim)
    while np.isnan(k[0]):
        search_n*=2
        k,dws,non_nan_points2,non_nan_values_bands=TS_IDW_4d(v,interp_indice,max_distance=max_distance*search_n,distence_w_dim=distence_w_dim)
    return k,dws,non_nan_points2,non_nan_values_bands
def zero(k):
    if k<0:
        return 0
    else:
        return k

In [3]:
#Locate validation points

for landcover_type in ['LCCS1','IGBP']:
    if landcover_type=='LCCS2':
        lc_len=10
    elif landcover_type=='LCCS3':
        lc_len=9
    elif landcover_type=='IGBP':
        lc_len=16
    elif landcover_type=='LCCS1':
        lc_len=15      
    aaa={'y':[],'m':[],'lon':[],'lat':[],'type':[]}
    landtypes=['landtype'+str(i) for i in range(1,lc_len+1)]
    for y in range(2001,2021):
        print(y)
        for m in range(1,13):
            f=torch.load(f"/data2/hzy/albedo2/albedo_information/result_{y}_{m}_all_{landcover_type}.pth")
            a=[]
            for k in f.keys():
                if f[k][0]==1 and f[k][1][-1]==f[k][1][-2]:
                    a.append(k)
            random.shuffle(a)
            a=a[:600]
            aa=[]
            for i in a:
                tt=[landtypes[j] for j in range(lc_len) if f[i][1][j]>2.5]
                for t in tt:
                    aaa['y'].append(y)
                    aaa['m'].append(m)
                    aaa['lon'].append(int(i.split(',')[1]))
                    aaa['lat'].append(int(i.split(',')[0]))
                    aaa['type'].append(t)
    landtypes=['snow']
    for y in range(2001,2021):
        print(y)
        for m in range(1,13):
            f=torch.load(f"/data2/hzy/albedo2/albedo_information/result_{y}_{m}_all_{landcover_type}.pth")
            a=[]
            for k in f.keys():
                if f[k][0]==1 and f[k][1][-4]>2.5:
                    if not np.isnan(f[k][2][lc_len,0]):
                        a.append(k)
            random.shuffle(a)
            a=a[:600]
            aa=[]
            for i in a:
                aaa['y'].append(y)
                aaa['m'].append(m)
                aaa['lon'].append(int(i.split(',')[1]))
                aaa['lat'].append(int(i.split(',')[0]))
                aaa['type'].append('snow')
    aaa_pd=pd.DataFrame.from_dict(aaa)
    aaa_pd.to_csv(f'/data2/hzy/albedo2/data/nan_coords_{landcover_type}.csv')

2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


In [4]:
#Add the SZA and remove validation points with a SZA greater than 85 degrees.
for landcover_type in ['LCCS1','IGBP']:
    aaa_pd=pd.read_csv(f'/data2/hzy/albedo2/data/nan_coords_{landcover_type}.csv')

    sza_ar,sza_L=torch.load(root_path+'sza_and_szaL.pth')
    sza_list=[]
    sza_L_list=[]
    for i in range(len(aaa_pd)):
        sza_L_list.append(sza_L[0,aaa_pd.iloc[i]['m']-1,90-aaa_pd.iloc[i]['lat'],0])
        sza_list.append(sza_ar[0,aaa_pd.iloc[i]['m']-1,90-aaa_pd.iloc[i]['lat'],0])
    aaa_pd['sza']=sza_list
    aaa_pd['sza_L']=sza_L_list
    aaa_pd=aaa_pd[aaa_pd['sza_L']<2]
    len(aaa_pd)
    aaa_pd.to_csv(f'/data2/hzy/albedo2/data/nan_coords_{landcover_type}.csv')

In [5]:
# Add ground truth for the validation points.
for landcover_type in ['LCCS1','IGBP']:
    if landcover_type=='LCCS2':
        lc_len=10
    elif landcover_type=='LCCS3':
        lc_len=9
    elif landcover_type=='IGBP':
        lc_len=16
    elif landcover_type=='LCCS1':
        lc_len=15   
    aaa_pd=pd.read_csv(f'/data2/hzy/albedo2/data/nan_coords_{landcover_type}.csv')
    aaa2=[]
    landtypes=['landtype'+str(i) for i in range(1,lc_len+1)]+['snow']
    bands=['albedo_BSA_vis', 
        'albedo_WSA_vis', 
        'albedo_BSA_nir', 
        'albedo_WSA_nir', 
        'albedo_BSA_shortwave', 
        'albedo_WSA_shortwave']
    for m in range(1,13):
        print(m)
        NC=nc.Dataset(f"/data2/hzy/albedo2/albedo_information_nc/month_{m}_{landcover_type}.nc",'r')
        for landtype in landtypes:
            kk={}
            aa_type=aaa_pd[(aaa_pd.type==landtype) & (aaa_pd.m==m)]
            n=0
            for band in bands:
                img=NC.variables[f'{band}-{landtype}'][:]
                # aa_type=[[y-2001,90-i[0],i[1]+180] for i in aaa[y] if i[2]==landtype]
                kk[band]=[]
                for ind, row in aa_type.iterrows():
                    kk[band].append(img[row['y']-2001,90-row['lat'],row['lon']+180])
                n+=1
                aa_type[f'main_band{n}']=kk[band]
            aaa2.append(aa_type)
        NC.close()
    aaa_pd2=pd.concat(aaa2)
    aaa_pd2.to_csv(f'/data2/hzy/albedo2/data/nan_coords2_{landcover_type}.csv')

1
2
3
4
5
6
7
8
9
10
11
12
1
2
3
4
5
6
7
8
9
10
11
12


In [ ]:
#Set the albedo of ALLUMs at the validation points to 2000.
for landcover_type in ['IGBP','LCCS1']:
    if landcover_type=='LCCS2':
        lc_len=10
    elif landcover_type=='LCCS3':
        lc_len=9
    elif landcover_type=='IGBP':
        lc_len=16
    elif landcover_type=='LCCS1':
        lc_len=15  
    aaa_pd=pd.read_csv(f'/data2/hzy/albedo2/data/nan_coords_{landcover_type}.csv')
    
    landtypes=['landtype'+str(i) for i in range(1,lc_len+1)]+['snow']
    bands=['albedo_BSA_vis', 
    'albedo_WSA_vis', 
    'albedo_BSA_nir', 
    'albedo_WSA_nir', 
    'albedo_BSA_shortwave',
    'albedo_WSA_shortwave']
    for m in range(1,13):
        print(m)
        NC=nc.Dataset(f"/data2/hzy/albedo2/albedo_information_nc/nan_to_2000/month_{m}_{landcover_type}.nc",'r+')
        for landtype in landtypes:
            for band in bands:
                img=NC.variables[f'{band}-{landtype}'][:]
                # aa_type=[[y-2001,90-i[0],i[1]+180] for i in aaa[y] if i[2]==landtype]
                aa_type=aaa_pd[(aaa_pd.type==landtype) & (aaa_pd.m==m)]
                for ind, row in aa_type.iterrows():
                    img[row['y']-2001,90-row['lat'],row['lon']+180]=2000
                NC.variables[f'{band}-{landtype}'][:]=img
                NC.sync()
        NC.close()

In [ ]:
#search for the 20 nearest neighboring grids to the missing value, 
# and return the albedo and distance vectors of the neighboring grids.
for landcover_type in ['LCCS1','IGBP']:
    if landcover_type=='LCCS2':
        lc_len=10
    elif landcover_type=='LCCS3':
        lc_len=9
    elif landcover_type=='IGBP':
        lc_len=16
    elif landcover_type=='LCCS1':
        lc_len=15  
    aaa_pd=pd.read_csv(f'/data2/hzy/albedo2/data/nan_coords2_{landcover_type}.csv')
    
    landtypes=['landtype'+str(i) for i in range(1,lc_len+1)]+['snow']
    all_pd_list=[]
    for month in range(1,13):
        print(month)
        NC=nc.Dataset(f"/data2/hzy/albedo2/albedo_information_nc/nan_to_2000/month_{month}_{landcover_type}.nc",'r+')
        for type in landtypes:
            v_fill,dws,pd_all=fill_albedo(month,type,landcover_type)
            all_pd_list.append(pd_all)
        NC.close()
    all_pd_concat=pd.concat(all_pd_list)
    all_pd_concat.to_csv(f'/data2/hzy/albedo2/data/regression_{landcover_type}.csv')
    # break
    print(len(all_pd_concat),len(aaa_pd))
    aaaaa=pd.merge(aaa_pd,all_pd_concat,how='outer')
    print(len(aaaaa))
    aaaaa.to_csv(f'/data2/hzy/albedo2/data/regression_{landcover_type}.csv')

In [10]:
#Regression distance weights
for landcover_type in ['LCCS1','LCCS2','LCCS3','IGBP']:
    if landcover_type=='LCCS2':
        lc_len=10
    elif landcover_type=='LCCS3':
        lc_len=9
    elif landcover_type=='IGBP':
        lc_len=16
    elif landcover_type=='LCCS1':
        lc_len=15  
    data=pd.read_csv(f'/data2/hzy/albedo2/data/regression_{landcover_type}.csv')
    print(len(data))
    data=data.dropna(how='any')
    print(len(data))
    ws_all={}
    landtypes=['landtype'+str(i) for i in range(1,lc_len+1)]
    if landcover_type in 'LCCS2':
        landtypes=['snow']+landtypes

    for lt in landtypes:
        if lt=='landtype2':
            continue
        print(lt)
        data_lt=data[(data.type==lt)&(data.sza_L==0)]
        res=[]
        for band_id in range(1,7):
            res.append(np.array(data_lt[f'main_band{band_id}'])-np.array(data_lt[f'sub_band{band_id}']))
        d_offset=[]
        for d in range(1,5):
            d_offset.append(np.array(data_lt[f'd{d}']))
        d_offset=np.stack(d_offset)
        d_offset_sq=d_offset**2
        res=np.stack(res)
        res_sq=abs(res)#这里应该是**(1/q）,再**2
        ws_b=[]
        for band_i in range(6):
            x,y=d_offset_sq.transpose((1,0)).copy(), res_sq[band_i].copy()
            model = LinearRegression(fit_intercept=False).fit(x,y)#无截距且回归系数限制大于0
            ws=model.coef_
            ws2=[math.sqrt(ws[i]/ws[-1]) if ws[i]>=0 else -1 for i in range(4)]
            ws_b.append(ws2)
        ws_b=np.stack(ws_b)
            # if -1 in ws:
        ws_all[lt]=ws_b
        print(ws_b)
    torch.save(ws_all,f'/data2/hzy/albedo2/data/weight_{landcover_type}.pth')

19867101
19504863
snow
[[2.2258064  1.34244627 1.27400105 1.        ]
 [2.21986013 1.34109813 1.27368293 1.        ]
 [2.39426556 1.38927213 1.26953466 1.        ]
 [2.30753605 1.38693972 1.26644574 1.        ]
 [2.28402223 1.35713011 1.27584479 1.        ]
 [2.24805117 1.35493525 1.27411601 1.        ]]
landtype1
[[0.85874887 0.72862314 1.05360882 1.        ]
 [0.81789288 0.69626545 1.05734151 1.        ]
 [0.61962165 0.59944437 1.08423188 1.        ]
 [0.62519667 0.60491252 1.08359046 1.        ]
 [0.70070264 0.63333449 1.0730686  1.        ]
 [0.69625144 0.63672626 1.07357515 1.        ]]
landtype3
[[1.85391385 1.02761583 1.06789421 1.        ]
 [1.64589891 0.9992926  1.06931583 1.        ]
 [1.03102482 0.75901882 1.06766686 1.        ]
 [1.10599224 0.77160403 1.0669748  1.        ]
 [1.2436406  0.81675948 1.0655941  1.        ]
 [1.18593322 0.82197539 1.06597578 1.        ]]
landtype4
[[0.99168634 0.83661486 0.97525658 1.        ]
 [0.97689905 0.80608402 0.98074604 1.        ]
 [0.